In [217]:
##### 1. Importe de librerías y lectura de los datos #####
# Manipulación de datos
import pandas as pd
import numpy as np
# Visualizar
import matplotlib.pyplot as plt
import seaborn as sns
# Escalar y PCA
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA

In [219]:
EmpleadosAttrition = pd.read_csv('empleadosRETO.csv')
EmpleadosAttrition.head(10)

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,...,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition
0,50,Travel_Rarely,Research & Development,1 km,2,Medical,1,997,4,Male,...,22,4,3,80,32,1,2,4,1,No
1,36,Travel_Rarely,Research & Development,6 km,2,Medical,1,178,2,Male,...,20,4,4,80,7,0,3,2,0,No
2,21,Travel_Rarely,Sales,7 km,1,Marketing,1,1780,2,Male,...,13,3,2,80,1,3,3,0,1,Yes
3,52,Travel_Rarely,Research & Development,7 km,4,Life Sciences,1,1118,2,Male,...,19,3,4,80,18,4,3,6,4,No
4,33,Travel_Rarely,Research & Development,15 km,1,Medical,1,582,2,Male,...,12,3,4,80,15,2,4,6,7,Yes
5,47,Travel_Rarely,Research & Development,4 km,3,Life Sciences,1,1827,3,Female,...,12,3,3,80,28,4,3,11,14,No
6,22,Travel_Rarely,Research & Development,1 km,2,Life Sciences,1,872,4,Male,...,22,4,1,80,4,2,1,2,2,No
7,33,Non-Travel,Sales,16 km,3,Life Sciences,1,1681,3,Female,...,25,4,3,80,7,2,3,5,1,No
8,40,Travel_Rarely,Sales,4 km,4,Marketing,1,215,3,Male,...,11,3,1,80,15,2,2,11,2,No
9,27,Travel_Frequently,Sales,2 km,1,Life Sciences,1,1371,4,Male,...,20,4,2,80,8,3,3,2,0,No


In [221]:
# analizamos las dimensiones del df
EmpleadosAttrition.shape

(400, 30)

In [223]:
EmpleadosAttrition.dtypes

Age                          int64
BusinessTravel              object
Department                  object
DistanceFromHome            object
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeNumber               int64
EnvironmentSatisfaction      int64
Gender                      object
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
MonthlyIncome                int64
NumCompaniesWorked           int64
HiringDate                  object
Over18                      object
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StandardHours                int64
TotalWorkingYears            int64
TrainingTimesLastYear        int64
WorkLifeBalance              int64
YearsInCurrentRole           int64
YearsSinceLastPromot

In [225]:
EmpleadosAttrition[['HiringDate','Over18','OverTime']]

,HiringDate,Over18,OverTime
0,06/06/2013,Y,No
1,12/25/2015,Y,No
2,2/14/2017,Y,No
3,7/29/2010,Y,No
4,10/07/2011,Y,Yes
...,...,...,...
395,05/09/2013,Y,Yes
396,04/02/2016,Y,Yes
397,1/21/2008,Y,Yes
398,8/27/2018,Y,No


In [227]:
EmpleadosAttrition[['HiringDate','Over18','OverTime']].nunique() # nos cercioramos de la cantidad de valores distintos que tienen

HiringDate    389
Over18          1
OverTime        2
dtype: int64

In [229]:
##### 2. Eliminar columnas irrelevantes #####
EmpleadosAttrition[['EmployeeCount','EmployeeNumber','Over18','StandardHours']].nunique() # nos cercioramos de la cantidad de valores distintos que tienen

EmployeeCount       1
EmployeeNumber    400
Over18              1
StandardHours       1
dtype: int64

In [231]:
EmpleadosAttrition = EmpleadosAttrition.drop(['EmployeeCount','EmployeeNumber','Over18','StandardHours'], axis=1)
EmpleadosAttrition.shape

(400, 26)

In [233]:
##### 3. Crear nuevas columnas a partir de 'HiringDate' #####
# Identificamos errores en fechas, por ejemplo, bisiestos en años incorrectos o fechas que no se sabe si tienen formato MM/DD/AAAA o DD/MM/AAAA
# La tendencia indica que el formato es MM/DD/AAAA. Intentamos forzar con "format='%m/%d/%Y'", pero no funcionó

EmpleadosAttrition['HiringDate'] = pd.to_datetime(
    EmpleadosAttrition['HiringDate'],
    format='%m/%d/%Y',
    errors='coerce'  # convertimos errores en NAT (null en fechas)
)

In [235]:
# Filas con error en la conversión de fechas
EmpleadosAttrition[EmpleadosAttrition['HiringDate'].isna()]

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,...,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition
229,36,Travel_Rarely,Sales,11 km,4,Marketing,2,Female,2,2,...,No,13,3,1,8,2,2,3,0,No


In [237]:
EmpleadosAttrition = EmpleadosAttrition.dropna(subset=['HiringDate']) # eliminamos los valores en que HiringDate = NaT 

In [239]:
EmpleadosAttrition['Year'] = pd.to_datetime(EmpleadosAttrition['HiringDate']).dt.year # convertimos a datetime y extraemos solo el year
EmpleadosAttrition['Year']

0      2013
1      2015
2      2017
3      2010
4      2011
       ... 
395    2013
396    2016
397    2008
398    2018
399    2010
Name: Year, Length: 399, dtype: int32

In [241]:
# Obtenemos los años que el empleado lleva en la empresa (hasta 2018)
EmpleadosAttrition['YearsAtCompany'] = 2018 - EmpleadosAttrition['Year']
EmpleadosAttrition['YearsAtCompany']

0       5
1       3
2       1
3       8
4       7
       ..
395     5
396     2
397    10
398     0
399     8
Name: YearsAtCompany, Length: 399, dtype: int32

In [243]:
##### 4. Limpieza de 'DistanceFromHome' que tiene string al final ("km") y eliminar columnas innecesarias
# Renombrar 'DistanceFromHome' como 'DistanceFromHome_km'
EmpleadosAttrition = EmpleadosAttrition.rename(columns={'DistanceFromHome':'DistanceFromHome_km'})
EmpleadosAttrition['DistanceFromHome_km']

0       1 km
1       6 km
2       7 km
3       7 km
4      15 km
       ...  
395    14 km
396    20 km
397    11 km
398     4 km
399    14 km
Name: DistanceFromHome_km, Length: 399, dtype: object

In [245]:
# Crear la nueva variable 'DistanceFromHome' como int
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome_km'].str.replace(" km","").astype(int)
EmpleadosAttrition['DistanceFromHome'].head()

0     1
1     6
2     7
3     7
4    15
Name: DistanceFromHome, dtype: int64

In [247]:
# Eliminar columnas que ya no se usan: 'Year', 'HiringDate', 'DistanceFromHome_km'
EmpleadosAttrition = EmpleadosAttrition.drop(['Year','HiringDate','DistanceFromHome_km'], axis=1)

In [249]:
##### 5. Generar frame informativo de sueldo promedio #####
# Hay que agrupar los departamentos; creamos el frame con estas columnas agrupadas
SueldoPromedioDepto = EmpleadosAttrition.groupby('Department')['MonthlyIncome'].mean().reset_index()
# aplicamos reset_index ya que hasta 'mean()', el departamento es el índice
# con este paso, deja de ser una serie y pasa a ser un df

In [251]:
SueldoPromedio = SueldoPromedioDepto
SueldoPromedio

,Department,MonthlyIncome
0,Human Resources,6239.888889
1,Research & Development,6804.149813
2,Sales,7192.609756


In [253]:
##### 6. Escalar 'MonthlyIncome' (que tenga valor entre 0 y 1) #####
# Aquí empleamos MinMaxScaler
escalador = MinMaxScaler() # creación del escalador

In [255]:
EmpleadosAttrition['MonthlyIncome'] = escalador.fit_transform(EmpleadosAttrition[['MonthlyIncome']])

In [257]:
EmpleadosAttrition['MonthlyIncome']

0      0.864269
1      0.207340
2      0.088062
3      0.497574
4      0.664470
         ...   
395    0.075248
396    0.187197
397    0.589327
398    0.121124
399    0.092122
Name: MonthlyIncome, Length: 399, dtype: float64

In [259]:
EmpleadosAttrition['MonthlyIncome'].min(), EmpleadosAttrition['MonthlyIncome'].max()

(0.0, 1.0)

In [261]:
##### 7. Convertir las categóricas a numéricas #####
# Identificamos que son categóricas nominales
# También identificar que nuestra variable de salida es dummy ('Attrition'), por lo que podríamos emplear LabelEncoder
le = LabelEncoder()
EmpleadosAttrition['Attrition'] = le.fit_transform(EmpleadosAttrition['Attrition'])

In [263]:
EmpleadosAttrition = pd.get_dummies(
    EmpleadosAttrition,
    columns=['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus'],
    drop_first=True # eliminamos la primer columna para evitar multicolinealidad
)

In [265]:
# Convertimos booleanos a números
bool_cols = EmpleadosAttrition.select_dtypes(include='bool').columns
EmpleadosAttrition[bool_cols] = EmpleadosAttrition[bool_cols].astype(int)

In [267]:
EmpleadosAttrition.head()

,Age,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,...,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single
0,50,2,4,3,4,4,0.864269,9,No,22,...,0,0,0,0,1,0,0,0,0,0
1,36,2,2,3,2,2,0.207340,6,No,20,...,0,0,0,1,0,0,0,0,0,0
2,21,1,2,3,1,2,0.088062,1,No,13,...,0,0,0,0,0,0,0,1,0,1
3,52,4,2,3,3,2,0.497574,7,No,19,...,0,0,0,0,0,0,0,0,0,1
4,33,1,2,3,3,3,0.664470,7,Yes,12,...,0,0,1,0,0,0,0,0,1,0


In [269]:
# Convertimos el binario que queda ('OverTime') a numérico
EmpleadosAttrition['OverTime'] = EmpleadosAttrition['OverTime'].map({'Yes': 1, 'No': 0})
EmpleadosAttrition.head()

,Age,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,...,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single
0,50,2,4,3,4,4,0.864269,9,0,22,...,0,0,0,0,1,0,0,0,0,0
1,36,2,2,3,2,2,0.207340,6,0,20,...,0,0,0,1,0,0,0,0,0,0
2,21,1,2,3,1,2,0.088062,1,0,13,...,0,0,0,0,0,0,0,1,0,1
3,52,4,2,3,3,2,0.497574,7,0,19,...,0,0,0,0,0,0,0,0,0,1
4,33,1,2,3,3,3,0.664470,7,1,12,...,0,0,1,0,0,0,0,0,1,0


In [271]:
##### 8. Evaluar la importancia a través de correlación con Attrition para filtrar variables #####
correlacion = EmpleadosAttrition.corr() # calculamos la matriz

In [273]:
correlacion_attrition = correlacion['Attrition'].abs() # conservamos la corr de 'Attrition' con el resto de variables

In [275]:
# Filtrar solo las columnas con corr > 0.1
columnas_seleccionadas = correlacion_attrition[correlacion_attrition >= 0.1].index # nos quedamos con aquellas que tenga corr
# >= 0.1 con respecto a 'Attrition

In [277]:
EmpleadosAttritionFinal = EmpleadosAttrition[columnas_seleccionadas] # generamos nuevo df con variables útiles

In [283]:
EmpleadosAttritionFinal.head()

,Age,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,OverTime,TotalWorkingYears,YearsInCurrentRole,Attrition,YearsAtCompany,EducationField_Technical Degree,JobRole_Laboratory Technician,JobRole_Research Director,JobRole_Sales Representative,MaritalStatus_Single
0,50,4,3,4,4,0.864269,0,32,4,0,5,0,0,1,0,0
1,36,2,3,2,2,0.207340,0,7,2,0,3,0,0,0,0,0
2,21,2,3,1,2,0.088062,0,1,0,1,1,0,0,0,1,1
3,52,2,3,3,2,0.497574,0,18,6,0,8,0,0,0,0,1
4,33,2,3,3,3,0.664470,1,15,6,1,7,0,0,0,0,0


In [285]:
##### 9. Aplicar PCA #####
X = EmpleadosAttritionFinal.drop('Attrition', axis=1) # separar a la variable de salida
Y = EmpleadosAttritionFinal['Attrition']

In [287]:
# Aplicamos PCA
pca = PCA()
X_pca = pca.fit_transform(X)

In [291]:
# Determinar el 80% que explica la varianza
varianza_acumulada = np.cumsum(pca.explained_variance_ratio_)

In [293]:
n_componentes = np.argmax(varianza_acumulada >= 0.80) + 1 # encontramos el primer "True" y sumamos 1 para modificar el índice
print("Componentes necesarios:", n_componentes)

Componentes necesarios: 2


In [297]:
EmpleadosAttritionFinal = EmpleadosAttrition[columnas_seleccionadas].copy()

for i in range(n_componentes):
    EmpleadosAttritionFinal[f'C{i}'] = X_pca[:, i]

In [301]:
EmpleadosAttritionFinal.head()

,Age,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,OverTime,TotalWorkingYears,YearsInCurrentRole,Attrition,YearsAtCompany,EducationField_Technical Degree,JobRole_Laboratory Technician,JobRole_Research Director,JobRole_Sales Representative,MaritalStatus_Single,C0,C1
0,50,4,3,4,4,0.864269,0,32,4,0,5,0,0,1,0,0,20.239766,4.210151
1,36,2,3,2,2,0.207340,0,7,2,0,3,0,0,0,0,0,-6.395229,3.698033
2,21,2,3,1,2,0.088062,0,1,0,1,1,0,0,0,1,1,-21.519763,-1.740765
3,52,2,3,3,2,0.497574,0,18,6,0,8,0,0,0,0,1,13.812963,5.840909
4,33,2,3,3,3,0.664470,1,15,6,1,7,0,0,0,0,0,-1.440168,-4.143749


In [303]:
##### 10. Guardamos en un csv #####
EmpleadosAttritionFinal.to_csv('EmpleadosAttritionFinal.csv', index=False)